## Install the tensorflow-datasets 

In [2]:
#  !pip install -q tensorflow-datasets

In [39]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [40]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [41]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [42]:
training_sentences = []
training_labels = []
testing_sentences= []
testing_labels = []

In [43]:
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

In [44]:
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [45]:
# For training the numpy arrays should be in the form of arrays
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [51]:
len(training_sentences)


25000

In [52]:
len(testing_sentences)

25000

## Hyperparameters

In [22]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok ="<OOV>"

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
# word_index is the list of words derived from the training set
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

### Defining the Neural Network

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


#### Often in NLP a different layer than Flatten() is used ie; the GlobalAveragePooling1D to have a smaller output being fed into the Dense layer

In [29]:
model= tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
    
])

As we can see from the below model summary, the global_average_pooling1d layer has a smaller output shape of 16. This is achieved by averaging across the vector to flatten it out

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [34]:
num_epochs = 10
model.fit(padded, 
          training_labels_final,
          epochs=num_epochs,
          verbose=2,
          validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 - 3s - loss: 0.1983 - accuracy: 0.9408 - val_loss: 1.0147 - val_accuracy: 0.8310
Epoch 2/10
25000/25000 - 3s - loss: 0.1870 - accuracy: 0.9457 - val_loss: 1.1278 - val_accuracy: 0.8271
Epoch 3/10
25000/25000 - 3s - loss: 0.1772 - accuracy: 0.9495 - val_loss: 1.2357 - val_accuracy: 0.8238
Epoch 4/10
25000/25000 - 3s - loss: 0.1739 - accuracy: 0.9540 - val_loss: 1.4604 - val_accuracy: 0.8130
Epoch 5/10
25000/25000 - 3s - loss: 0.1703 - accuracy: 0.9575 - val_loss: 1.4823 - val_accuracy: 0.8152
Epoch 6/10
25000/25000 - 3s - loss: 0.1649 - accuracy: 0.9632 - val_loss: 1.5326 - val_accuracy: 0.8171
Epoch 7/10
25000/25000 - 3s - loss: 0.1541 - accuracy: 0.9676 - val_loss: 1.6964 - val_accuracy: 0.8155
Epoch 8/10
25000/25000 - 3s - loss: 0.1462 - accuracy: 0.9724 - val_loss: 1.7704 - val_accuracy: 0.8143
Epoch 9/10
25000/25000 - 3s - loss: 0.1363 - accuracy: 0.9770 - val_loss: 1.8819 - val_accuracy: 0.8121
Epoch 10/10
25

# Visualizing the Embeddings

In [35]:
#First get the results from the embedding layer which is layer zero
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [37]:
# To be able to plot is we need a helper function to be able to revese our word
# index
# Example:
# Hello: 1, World: 2, How: 3
# to
# 1: Hello, 2: World, 3: How
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

## Write the vectors and their meta data out to file

In [38]:
import io
# The tensorflow projector uses this file type to plot the vectors in 3D space
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write("\t".join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

#### To render the results go to "projector.tensorflow.org" 
#### Load the data from the computer, ie; for the 1st load the vecs.tsv
#### for the 2nd load the meta.tsv

#### The embeddings projector --> https://projector.tensorflow.org/